# Sentinel-5P


This notebook explores working with Sentinel-3 data. 

## Sentinel-5P OLCI

https://sentinels.copernicus.eu/web/sentinel/user-guides/sentinel-3-olci

The OLCI dataset provided by Sentinelhub is based on the level-1b products. These products are provided in "instrument" projection rather than being projected into a ground-based reference system. Hence, these products do not come with a 'native' reference system. The openEO collections are currently configured to use EPSG:4326 unprojected coordinates, with a resolution set to a fixed value that tries to approximate the native 300m ground resolution.


In [1]:
import openeo
import xarray
import matplotlib.pyplot as plt
import numpy as np

In [3]:
conn = openeo.connect("openeo.dataspace.copernicus.eu")
conn.authenticate_oidc()

Authenticated using refresh token.


<Connection to 'https://openeo.dataspace.copernicus.eu/openeo/1.2/' with OidcBearerAuth>

### Load the collection

In [4]:
conn.describe_collection("SENTINEL_5P_L2")

{'cube:dimensions': {'bands': {'type': 'bands',
   'values': ['CO',
    'HCHO',
    'NO2',
    'O3',
    'SO2',
    'CH4',
    'AER_AI_340_380',
    'AER_AI_354_388',
    'CLOUD_BASE_PRESSURE',
    'CLOUD_TOP_PRESSURE',
    'CLOUD_BASE_HEIGHT',
    'CLOUD_TOP_HEIGHT',
    'CLOUD_OPTICAL_THICKNESS',
    'CLOUD_FRACTION',
    'dataMask']},
  't': {'extent': ['2018-04-30T00:18:50Z', None],
   'step': 'P1D',
   'type': 'temporal'},
  'x': {'axis': 'x',
   'extent': [-180, 180],
   'reference_system': 4326,
   'type': 'spatial'},
  'y': {'axis': 'y',
   'extent': [-85, 85],
   'reference_system': 4326,
   'type': 'spatial'}},
 'description': 'Sentinel 5 Precursor imagery captured by TROPOMI sensor.\n\nThis dataset only supports loading one band at a time.',
 'extent': {'spatial': {'bbox': [[-180.0, -85.0, 180.0, 85.0]]},
  'temporal': {'interval': [['2018-04-30T00:18:50Z', None]]}},
 'id': 'SENTINEL_5P_L2',
 'license': 'proprietary',
 'links': [{'href': 'https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/',
   'rel': 'root',
   'type': 'application/json'},
  {'href': 'https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/collections/sentinel-5p-l2',
   'rel': 'self',
   'type': 'application/json'},
  {'href': 'https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/collections',
   'rel': 'parent',
   'type': 'application/json'},
  {'href': 'https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/collections/sentinel-5p-l2/queryables',
   'rel': 'http://www.opengis.net/def/rel/ogc/1.0/queryables',
   'type': 'application/schema+json'},
  {'href': 'https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/collections/sentinel-5p-l2/items',
   'rel': 'items',
   'type': 'application/geo+json'}],
 'providers': [],
 'sci:citation': 'Modified Copernicus Sentinel data [Year]/Sentinel Hub',
 'stac_extensions': ['https://stac-extensions.github.io/scientific/v1.0.0/schema.json',
  'https://stac-extensions.github.io/sat/v1.0.0/schema.json',
  'https://docs.sentinel-hub.com/api/latest/stac/s5p/v1.0.0/schema.json'],
 'stac_version': '1.0.0',
 'summaries': {'eo:bands': [{'data_type': 'float32',
    'description': 'Carbon monoxide',
    'name': 'CO',
    'openeo:gsd': {'unit': '°',
     'value': [0.054563492063483, 0.034722222222216]},
    'statistics': {'maximum': [0.1], 'minimum': [0]}},
   {'data_type': 'float32',
    'description': 'Formaldehyde',
    'name': 'HCHO',
    'openeo:gsd': {'unit': '°',
     'value': [0.054563492063483, 0.034722222222216]},
    'statistics': {'maximum': [0.001], 'minimum': [0]}},
   {'data_type': 'float32',
    'description': 'Nitrogen oxide',
    'name': 'NO2',
    'openeo:gsd': {'unit': '°',
     'value': [0.054563492063483, 0.034722222222216]},
    'statistics': {'maximum': [0.0003], 'minimum': [0]}},
   {'data_type': 'float32',
    'description': 'Ozone',
    'name': 'O3',
    'openeo:gsd': {'unit': '°',
     'value': [0.054563492063483, 0.034722222222216]},
    'statistics': {'maximum': [0.36], 'minimum': [0]}},
   {'data_type': 'float32',
    'description': 'Sulphur dioxide',
    'name': 'SO2',
    'openeo:gsd': {'unit': '°',
     'value': [0.054563492063483, 0.034722222222216]},
    'statistics': {'maximum': [0.01], 'minimum': [0]}},
   {'data_type': 'float32',
    'description': 'Methane',
    'name': 'CH4',
    'openeo:gsd': {'unit': '°',
     'value': [0.054563492063483, 0.034722222222216]},
    'statistics': {'maximum': [2000], 'minimum': [1600]}},
   {'data_type': 'float32',
    'description': 'UV (Ultraviolet) Aerosol Index calculated based on wavelengths of 340 nm and 380 nm',
    'name': 'AER_AI_340_380',
    'openeo:gsd': {'unit': '°',
     'value': [0.054563492063483, 0.034722222222216]},
    'statistics': {'maximum': [5], 'minimum': [-1]}},
   {'data_type': 'float32',
    'description': 'UV (Ultraviolet) Aerosol Index calculated based on wavelengths of 354 nm and 388 nm',
    'name': 'AER_AI_354_388',
    'openeo:gsd': {'unit': '°',
     'value': [0.054563492063483, 0.034722222222216]},
   

In [5]:
bbox = {"west": -66, "south": 14, "east": -58, "north": 18}
sentinel5_so2 = conn.load_collection(
    "SENTINEL_5P_L2",
    spatial_extent=bbox,
    temporal_extent=["2021-04-01", "2021-05-30"],
    bands=["SO2"],
)

sentinel5_aerosol_354 = conn.load_collection(
    "SENTINEL_5P_L2",
    spatial_extent=bbox,
    temporal_extent=["2021-04-01", "2021-05-30"],
    bands=["AER_AI_354_388"],
)

sentinel5_aerosol_340 = conn.load_collection(
    "SENTINEL_5P_L2",
    spatial_extent=bbox,
    temporal_extent=["2021-04-01", "2021-05-30"],
    bands=["AER_AI_340_380"],
)

Let's download this slice of data in netCDF format to give it an initial inspection.

In [6]:
sentinel5_so2.download("sentinel5_SO2.nc")
sentinel5_aerosol_354.download("sentinel5_aerosol_354.nc")
sentinel5_aerosol_340.download("sentinel5_aerosol_340.nc")